<a href="https://colab.research.google.com/github/saimasharleen/A-Visual-Search-Engine-for-Image-Retrieval-using-Feature-Distance-Metrics/blob/main/M02_resnet50-TL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array
import random
from tensorflow.keras.preprocessing import image

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')

print('TensorFlow version:', tf.__version__)
print('Is using GPU?', False if tf.config.list_physical_devices('GPU') == [] else True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# print(tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.12.0
Is using GPU? False
Num GPUs Available:  0


In [2]:
#connecting Google Drive with Colab
from google.colab import drive
import zipfile
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print('TensorFlow version:', tf.__version__)
print('Is using GPU?', False if tf.config.list_physical_devices('GPU') == [] else True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# print(tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.12.0
Is using GPU? False
Num GPUs Available:  0


In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [5]:
from glob import glob


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
train_path = '/content/gdrive/MyDrive/ML/dataset/Data/training'

In [8]:
import os
from google.colab import drive
drive.mount("/content/gdrive")


train_path = '/content/gdrive/MyDrive/ML/dataset/Data/training'

# List all subdirectories in the train_path directory
subdirectories = [f.path for f in os.scandir(train_path) if f.is_dir()]

# Iterate over the subdirectories and print their contents
for subdir in subdirectories:
    print(f"Subdirectory: {subdir}")
    files = os.listdir(subdir)
    for file in files:
        print(f"  - {file}")
    print()


Mounted at /content/gdrive
Subdirectory: /content/gdrive/MyDrive/ML/dataset/Data/training/will_smith
  - 800px-Bright_Japan_Premiere_Red_Carpet-_Will_Smith_(38627390265).jpg
  - 636px-Will_Smith_2.jpg
  - David_Ayer_&_Will_Smith_(28316583290).jpg
  - Will_Smith_Cannes_2017_2.jpg
  - 800px-Time_100_Will_Smith_title.jpg
  - 400px-Will_Smith_in_Berlin_(2011).jpg
  - 800px-Will_Smith_(35302305554).jpg
  - Will_Smith_2,_2012.jpg
  - 577px-Will_Smith_2011,_3.jpg
  - 538px-Will_Smith_Cannes_2017_3.jpg
  - Steve Lyon.jpg
  - 800px-Gazing_adoringly_at_Will_Smith.jpg
  - 543px-Will_Smith_Nobel_Peace_Prize_2009_Harry_Wad1.jpg
  - SDCC_2015_-_Margot_Robbie,_Will_Smith_&_David_Ayer_(19682725846).jpg
  - Will_Smith_Cannes_2017_4.jpg
  - 398px-Will_Smith_by_Gage_Skidmore.jpg

Subdirectory: /content/gdrive/MyDrive/ML/dataset/Data/training/kate_beckinsale
  - 381px-Kate_Beckinsale_(29907748884)_(cropped).jpg
  - 405px-Kate_Beckinsale_TIFF,_2008.jpg
  - 446px-KateBeckinsaleJune07.jpg
  - 692px-Kate_Beck

In [9]:
# useful for getting number of classes
folders = glob(train_path+'/*')

print(len(folders))

149


In [10]:
# add preprocessing layer to the front of ResNet50
resnet50 = tf.keras.applications.ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train ResNet50 weights
for layer in resnet50.layers:
  layer.trainable = False

resnet50.summary()

94765736/94765736 [==============================] - 4s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

In [11]:
# our layers - you can add more if you want
x = Flatten()(resnet50.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet50.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [12]:
# Make last 4 layers trainable
model.get_layer('conv5_block3_3_conv').trainable = True
model.get_layer('conv5_block3_2_conv').trainable = True
model.get_layer('conv5_block3_1_conv').trainable = True
model.get_layer('conv5_block2_3_conv').trainable = True

In [13]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.3)

# test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training')

test_set = train_datagen.flow_from_directory(train_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                            subset='validation')



In [15]:
from keras.callbacks import LearningRateScheduler
def lr_scheduler(epoch, lr):
    decay_rate = 0.85
    decay_step = 1
    if epoch % decay_step == 0 and epoch:
        return lr * pow(decay_rate, np.floor(epoch / decay_step))
    return lr

In [ ]:
# fit the model
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3),
        tf.keras.callbacks.ModelCheckpoint(
            'models/model_RESNET50_TL_{val_accuracy:.3f}.h5',
            save_best_only=True, save_weights_only=False,
            monitor='val_accuracy'
        ),
        LearningRateScheduler(lr_scheduler, verbose=1)
    ]
)

In [ ]:
# accuracies
accs = r.history['accuracy']
val_accs = r.history['val_accuracy']
loss = r.history['loss']
val_loss = r.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, accs, label='Training Accuracy')
plt.plot(epochs_range, val_accs, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()